# Monte Carlo Dispersion Analysis with the Dispersion Class


Finally the Monte Carlo simulations can be performed using a dedicated class called Dispersion.
Say goodbye to the long and tedious process of creating the Monte Carlo Simulations throughout jupyter notebooks!


In [1]:
%load_ext autoreload
%autoreload 2

First, let's import the necessary libraries, including the newest Dispersion class!


In [2]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, Dispersion
from rocketpy.monte_carlo import (
    McEnvironment,
    McSolidMotor,
    McRocket,
    McFlight,
    McNoseCone,
    McTail,
    McTrapezoidalFins,
    McParachute,
)
import datetime


If you are using Jupyter Notebooks, it is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.


In [3]:
%matplotlib inline

The Dispersion class allows us to perform Monte Carlo Simulations in a very simple way.
We just need to create an instance of the class, and then call the method `run_dispersion()` to perform the simulations.
The class has a lot of capabilities, but we will only use a few of them in this example.
We encourage you to check the documentation of the class to learn more about the Dispersion.

Also, you can check RocketPy's main reference for a better conceptual understanding
of the Monte Carlo Simulations: [RocketPy: Six Degree-of-Freedom Rocket Trajectory Simulator](<https://doi.org/10.1061/(ASCE)AS.1943-5525.0001331>)

TODO: improve the description


## First Step: Creating the Inputs for the Simulations


### Environment


In [4]:
env = Environment(
    railLength=5.7, latitude=39.389700, longitude=-8.288964, elevation=113
)

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

env.setDate((tomorrow.year, tomorrow.month, tomorrow.day, 12))  # Hour given in UTC time

env.setAtmosphericModel(type="Forecast", file="GFS")


TODO: Improve docs
Here we only add a standard deviation to rail length of 0.005


In [5]:
mc_env = McEnvironment(
    environment=env,
    railLength=0.005,
)

mc_env

# TODO: improve the print or __repr__() of these Monte Carlo classes


McEnvironment(railLength=(5.7, 0.005), date=[datetime.datetime(2023, 3, 11, 12, 0, tzinfo=<UTC>)], elevation=(141.80076211248732, 0), gravity=(9.80665, 0), latitude=(39.3897, 0), longitude=(-8.288964, 0), datum=['SIRGAS2000'], timeZone=['UTC'])

### Motor


Let's define the motor using the firs method. We will be using the data from the manufacturer, and following
the [RocketPy's documentation](https://docs.rocketpy.org/en/latest/user/index.html).


In [6]:
motor = SolidMotor(
    thrustSource="../../../data/motors/Cesaroni_M1670.eng",
    burnOutTime=3.9,
    grainsCenterOfMassPosition=-0.85704,
    grainNumber=5,
    grainSeparation=5 / 1000,
    grainDensity=1815,
    grainOuterRadius=33 / 1000,
    grainInitialInnerRadius=15 / 1000,
    grainInitialHeight=120 / 1000,
    nozzleRadius=33 / 1000,
    throatRadius=11 / 1000,
    interpolationMethod="linear",
    nozzlePosition=-1.255,
    coordinateSystemOrientation="nozzleToCombustionChamber",
)


In [19]:
v = [field_name for field_name, field_type in mc_motor.__fields__.items() if not field_type.required]
v

['thrustSource',
 'burnOutTime',
 'grainsCenterOfMassPosition',
 'grainNumber',
 'grainDensity',
 'grainOuterRadius',
 'grainInitialInnerRadius',
 'grainInitialHeight',
 'grainSeparation',
 'totalImpulse',
 'nozzleRadius',
 'nozzlePosition',
 'throatRadius']

In [15]:
a = {"1": 1,"j": 2}
for i, j in a:
    print(i,j)

ValueError: not enough values to unpack (expected 2, got 1)

In [18]:
for name, val in mc_motor.__fields__.items():
    print(name, val.required)

solidMotor True
thrustSource False
burnOutTime False
grainsCenterOfMassPosition False
grainNumber False
grainDensity False
grainOuterRadius False
grainInitialInnerRadius False
grainInitialHeight False
grainSeparation False
totalImpulse False
nozzleRadius False
nozzlePosition False
throatRadius False


In [7]:
mc_motor = McSolidMotor(
    solidMotor=motor,
    burnOutTime=1,
    grainsCenterOfMassPosition=0.001,
    grainDensity=50,
    grainInitialHeight=0.001,
    totalImpulse=100,
    throatRadius=0.0005,
    nozzlePosition=0.001,
)

mc_motor


{'solidMotor': ModelField(name='solidMotor', type=SolidMotor, required=True), 'thrustSource': ModelField(name='thrustSource', type=List[Union[FilePath, NoneType]], required=False, default=[]), 'burnOutTime': ModelField(name='burnOutTime', type=Optional[Any], required=False, default=0), 'grainsCenterOfMassPosition': ModelField(name='grainsCenterOfMassPosition', type=Optional[Any], required=False, default=0), 'grainNumber': ModelField(name='grainNumber', type=List[Union[StrictInt, StrictFloat, NoneType]], required=False, default=[]), 'grainDensity': ModelField(name='grainDensity', type=Optional[Any], required=False, default=0), 'grainOuterRadius': ModelField(name='grainOuterRadius', type=Optional[Any], required=False, default=0), 'grainInitialInnerRadius': ModelField(name='grainInitialInnerRadius', type=Optional[Any], required=False, default=0), 'grainInitialHeight': ModelField(name='grainInitialHeight', type=Optional[Any], required=False, default=0), 'grainSeparation': ModelField(name='

McSolidMotor(thrustSource=['../../../data/motors/Cesaroni_M1670.eng'], burnOutTime=(3.9, 1), grainsCenterOfMassPosition=(-0.85704, 0.001), grainNumber=[5], grainDensity=(1815, 50), grainOuterRadius=(0.033, 0), grainInitialInnerRadius=(0.015, 0), grainInitialHeight=(0.12, 0.001), grainSeparation=(0.005, 0), totalImpulse=(6026.35, 100), nozzleRadius=(0.033, 0), nozzlePosition=(-1.255, 0.001), throatRadius=(0.011, 0.0005))

### Rocket


In [8]:
rocket = Rocket(
    radius=127 / 2000,
    mass=19.197 - 2.956,
    inertiaI=6.60,
    inertiaZ=0.0351,
    powerOffDrag="../../../data/calisto/powerOffDragCurve.csv",
    powerOnDrag="../../../data/calisto/powerOnDragCurve.csv",
    centerOfDryMassPosition=0,
    coordinateSystemOrientation="tailToNose",
)

rocket.setRailButtons([0.2, -0.5])

rocket.addMotor(motor, position=-1.255)

nose_cone = rocket.addNose(length=0.55829, kind="vonKarman", position=1.278)

fin_set = rocket.addTrapezoidalFins(
    n=4,
    rootChord=0.120,
    tipChord=0.040,
    span=0.100,
    position=-1.04956,
)

tail = rocket.addTail(
    topRadius=0.0635, bottomRadius=0.0435, length=0.060, position=-1.194656
)


Additionally, we set parachutes for our Rocket, as well as the trigger functions for the deployment of such parachutes.


In [9]:
def drogueTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def mainTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate main when vz < 0 m/s and z < 500 + 100 m (+100 due to surface elevation).
    return True if y[5] < 0 and y[2] < 500 + 100 else False


main_chute = rocket.addParachute(
    "Main",
    CdS=10.0,
    trigger=mainTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

drogue_chute = rocket.addParachute(
    "Drogue",
    CdS=1.0,
    trigger=drogueTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)


In [10]:
mc_rocket = McRocket(
    rocket=rocket,
    radius=1,
    mass=1,
    inertiaZ=1,
    powerOffDragFactor=(1, 0.03),
    powerOnDragFactor=(1, 0.03),
)
mc_rocket


McRocket(radius=(0.0635, 1), mass=(16.241, 1), inertiaI=(6.6, 0), inertiaZ=(0.0351, 1), powerOffDrag=[Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power Off)], powerOnDrag=[Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power On)], centerOfDryMassPosition=(0, 0), powerOffDragFactor=(1, 0.03), powerOnDragFactor=(1, 0.03))

In [11]:
mc_nose_cone = McNoseCone(
    nosecone=nose_cone,
    length=0.001,
)

mc_fin_set = McTrapezoidalFins(
    trapezoidalFins=fin_set,
    rootChord=0.001,
    tipChord=0.001,
    span=0.001,
)

mc_tail = McTail(
    tail=tail,
    topRadius=0.001,
    bottomRadius=0.001,
    length=0.001,
)

mc_main = McParachute(
    parachute=main_chute,
    CdS=0.1,
    lag=0.1,
)

mc_drogue = McParachute(
    parachute=drogue_chute,
    CdS=0.1,
    lag=0.1,
)


In [20]:
mc_rocket.addMotor(mc_motor,position=0.001)
mc_rocket.addNose(mc_nose_cone,position=(1.278,0.001))
mc_rocket.addTrapezoidalFins(mc_fin_set,position=(0.001,"dist_func"))
mc_rocket.addTail(mc_tail,position=(-1.194656,0.001,"dist_func"))
mc_rocket.addParachute(mc_main)
mc_rocket.addParachute(mc_drogue)


In [21]:
print(mc_rocket.motors[0].position)
print(mc_rocket.nosecones[0].position)
print(mc_rocket.fins[0].position)
print(mc_rocket.tails[0].position)


(-1.255, 0.001)
(1.278, 0.001)
(-1.04956, 0.001, 'dist_func')
(-1.194656, 0.001, 'dist_func')


In [13]:
from typing import Any, List, Tuple, Union

from pydantic import BaseModel, Extra, Field, FilePath, StrictFloat, StrictInt, root_validator


class McSolidMotor(BaseModel):
    """_summary_

    Parameters
    ----------
    BaseModel : _type_
        _description_

    Returns
    -------
    _type_
        _description_

    Raises
    ------
    ValueError
        _description_
    """

    solidMotor: SolidMotor = Field(..., repr=False)
    thrustSource: List[Union[FilePath, None]] = []
    burnOutTime: Any = 0
    grainsCenterOfMassPosition: Any = 0
    grainNumber: List[Union[Union[StrictInt, StrictFloat], None]] = []
    grainDensity: Any = 0
    grainOuterRadius: Any = 0
    grainInitialInnerRadius: Any = 0
    grainInitialHeight: Any = 0
    grainSeparation: Any = 0
    totalImpulse: Any = 0
    nozzleRadius: Any = 0
    nozzlePosition: Any = 0
    throatRadius: Any = 0
    # TODO: why coordinateSystemOrientation is not included in this class?

    class Config:
        arbitrary_types_allowed = True
        extra=Extra.allow      

    @root_validator(skip_on_failure=True)
    def set_attr(cls, values):
        """Validates inputs that can be either tuples, lists, ints or floats and
        saves them in the format (nom_val,std) or (nom_val,std,'dist_func').
        Lists are saved as they are inputted.
        Inputs can be given as floats or ints, referring to the standard deviation.
        In this case, the nominal value of that attribute will come from the rocket
        object passed. If the distribution function needs to be specified, then a
        tuple with the standard deviation as the first item, and the string containing
        the name a numpy.random distribution function can be passed.
        If a tuple with a nominal value and a standard deviation is passed, then it
        will take priority over the rocket object attribute's value. A third item
        can also be added to the tuple specifying the distribution function"""

        validate_fields = [
            "thrustSource",
            "burnOutTime",
            "grainsCenterOfMassPosition",
            "grainNumber",
            "grainDensity",
            "grainOuterRadius",
            "grainInitialInnerRadius",
            "grainInitialHeight",
            "grainSeparation",
            "nozzleRadius",
            "nozzlePosition",
            "throatRadius",
            "totalImpulse",
        ]
        for field in validate_fields:
            v = values[field]
            # checks if tuple
            if isinstance(v, tuple):
                # checks if first item is valid
                assert isinstance(
                    v[0], (int, float)
                ), f"\nField '{field}': \n\tFirst item of tuple must be either an int or float"
                # if len is two can either be (nom_val,std) or (std,'dist_func')
                if len(v) == 2:
                    # checks if second value is either string or int/float
                    assert isinstance(
                        v[1], (int, float, str)
                    ), f"\nField '{field}': \n\tSecond item of tuple must be either an int, float or string \n\tIf the first value refers to the nominal value of {field}, then the second item's value should be the desired standard deviation \n\tIf the first value is the standard deviation, then the second item's value should be a string containing a name of a numpy.random distribution function"
                    # if second item is not str, then (nom_val, std)
                    if not isinstance(v[1], str):
                        values[field] = v
                    # if second item is str, then (nom_val, std, str)
                    else:
                        values[field] = (
                            getattr(values["solidMotor"], field),
                            v[0],
                            v[1],
                        )
                # if len is three, then (nom_val, std, 'dist_func')
                if len(v) == 3:
                    assert isinstance(
                        v[1], (int, float)
                    ), f"\nField '{field}': \n\tSecond item of tuple must be either an int or float \n\tThe second item should be the standard deviation to be used in the simulation"
                    assert isinstance(
                        v[2], str
                    ), f"\nField '{field}': \n\tThird item of tuple must be a string \n\tThe string should contain the name of a valid numpy.random distribution function"
                    values[field] = v
            elif isinstance(v, list):
                # checks if input list is empty, meaning nothing was inputted
                # and values should be gotten from class
                if len(v) == 0:
                    values[field] = [getattr(values["solidMotor"], field)]
                else:
                    # guarantee all values are valid (ints or floats)
                    assert all(
                        isinstance(item, (int, float)) for item in v
                    ), f"\nField '{field}': \n\tItems in list must be either ints or floats"
                    # all good, sets inputs
                    values[field] = v
            elif isinstance(v, (int, float)):
                # not list or tuple, must be an int or float
                # get attr and returns (nom_value, std)
                values[field] = (getattr(values["solidMotor"], field), v)
            else:
                raise ValueError(
                    f"\nField '{field}': \n\tMust be either a tuple, list, int or float"
                )
        return values


In [18]:
mc_motor = McSolidMotor(
    solidMotor=motor,
    burnOutTime=1,
    grainsCenterOfMassPosition=0.001,
    grainDensity=50,
    grainInitialHeight=0.001,
    totalImpulse=100,
    throatRadius=0.0005,
    nozzlePosition=0.001,
)
mc_motor.position = 12

In [19]:
print(mc_motor)

thrustSource=['../../../data/motors/Cesaroni_M1670.eng'] burnOutTime=(3.9, 1) grainsCenterOfMassPosition=(-0.85704, 0.001) grainNumber=[5] grainDensity=(1815, 50) grainOuterRadius=(0.033, 0) grainInitialInnerRadius=(0.015, 0) grainInitialHeight=(0.12, 0.001) grainSeparation=(0.005, 0) totalImpulse=(6026.35, 100) nozzleRadius=(0.033, 0) nozzlePosition=(-1.255, 0.001) throatRadius=(0.011, 0.0005) position=12


### Flight


In [22]:
test_flight = Flight(
    rocket=rocket,
    environment=env,
    inclination=84,
    heading=133,
)


In [23]:
mc_flight = McFlight(
    flight=test_flight,
    inclination=(84, 1),
    heading=(133, 1),
)


And we can visualize the flight trajectory:


In [24]:
# test_flight.plots.trajectory_3d()


### Starting the Monte Carlo Simulations


First, let's invoke the Dispersion class, we only need a filename to initialize it.
The filename will be used either to save the results of the simulations or to load them
from a previous ran simulation.


In [25]:
test_dispersion = Dispersion(
    filename="dispersion_analysis_outputs/disp_class_example",
    environment=mc_env,
    rocket=mc_rocket,
    flight=mc_flight,
)

# TODO: add custom warning o when the rocket doesn't have a motors, parachute, or aerosurfaces


Then, we can run the simulations using the method Dispersion.run_dispersion().
But before that, we need to set some simple parameters for the simulations.
We will set them by using a dictionary, which is one of the simplest way to do it.


Finally, let's iterate over the simulations and export the data from each flight simulation!


In [26]:
test_dispersion.run_dispersion(
    number_of_simulations=50,
    append=False,
)

# TODO: it seems the dispersion is falling into an eternal loop. Need to check it out 


'Starting'

KeyboardInterrupt: 

In [27]:
# In case you want to verify the new dispersion dictionary format...
test_dispersion.dispersion_dictionary


{'environment': {'railLength': (5.7, 0.005, <function RandomState.normal>),
  'date': [datetime.datetime(2023, 3, 6, 12, 0, tzinfo=<UTC>)],
  'elevation': (141.80076211248732, 0, <function RandomState.normal>),
  'gravity': (9.80665, 0, <function RandomState.normal>),
  'latitude': (39.3897, 0, <function RandomState.normal>),
  'longitude': (-8.288964, 0, <function RandomState.normal>),
  'datum': ['SIRGAS2000'],
  'timeZone': ['UTC']},
 'rocket': {'radius': (0.0635, 1, <function RandomState.normal>),
  'mass': (16.241, 1, <function RandomState.normal>),
  'inertiaI': (6.6, 0, <function RandomState.normal>),
  'inertiaZ': (0.0351, 1, <function RandomState.normal>),
  'powerOffDrag': [Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power Off)],
  'powerOnDrag': [Function from R1 to R1 : (Mach Number) → (Drag Coefficient with Power On)],
  'centerOfDryMassPosition': (0, 0, <function RandomState.normal>),
  'powerOffDragFactor': (1, 0.03, <function RandomState.normal>),
  

### Visualizing the results


Now we finally have the results of our Monte Carlo simulations loaded!
Let's play with them.


First, we can print numerical information regarding the results of the simulations.


In [ ]:
test_dispersion.import_results()


In [ ]:
test_dispersion.print_results()


Also, we can visualize histograms of such results


In [ ]:
test_dispersion.allInfo()


Export to kml so it can be visualized in Google Earth


In [ ]:
test_dispersion.exportEllipsesToKML(
    filename="dispersion_analysis_outputs/disp_class_example.kml",
    origin_lat=env.latitude,
    origin_lon=env.longitude,
    type="impact",
)
